# 回归Regression

# 1. Udacity内容
## Regression:   
- Supervised Learning: Take examples of inputs and outputs. Now, given a new input, predict its output. 
- Mapping continous inputs to outputs. (discrete, continous)

## Regression & Function Approximation:
例子child Height和函数逼近或回归有什么关系呢？
向平均值回落的回归概念与逼近函数、输入到输出的映射有什么关系？

例子Child Height是19世纪后期的研究成果，它将所有这些不同的量（父母的身高，人群的平均身高，目标子女的身高）巧妙地联系了起来。   
即用这种函数的方式在它们之间创建了关系。  
于是人们说“这真不错，我们要使用回归这个概念”（Regression to Mean）。    
但他们开始使用后，真正的用意并非向均数回归，而是基于一堆测量点寻找一种数学关系，所以这个词在后来被误用了。  
而我们现在依然在使用这个词。   
所以，现在**回归并不是指向平均值回落，而是使用函数形式来逼近一堆数据点。**


## Regression in Machine Learning:

## Finding the best constant function:
找到可以逼近所有点的最佳的常数。   
**常数函数**：$f(x) = c$    
**误差函数**：$E(c) = \sum_{i=1}^{n}(y_{i} - c)^{2}$ ，LOSS ERROR.    
想要求使得LOSS ERROR最小的常数项，对误差函数求导即可。   
**求导计算过程**：
    $\frac{dE(c)}{dc} = \sum_{i=1}^{n} 2*(y_{i} -c)*(-1)$     
想要找到最小值，使得上式等于0即可。   
    即$\sum_{i=1}^{n} 2*(y_{i} -c)*(-1) = 0$    
    $-\sum_{i=1}^{n}2*(y_{i}-c) = 0$    
    $\sum_{i=1}^{n}y_{i} - \sum_{i=1}^{n}c=0$   
    $n*c = \sum_{i=1}^{n}y_{i}$    
    $c = \frac{\sum_{i=1}^{n}y_{i}}{c}$
    
 根据计算结果，最佳的常数是所有y的平均值。
 
 ## Order of Polynomial多项式的阶：
 k=0: constant    
 k=1: line     
 k=2: parabola    
     $f(x) = c_{0} + c_{1}*x + c_{2}*x^{2} + ... + c_{k}*x^{k}$  
 
 ![title](../images/regression_1.png)
 
 在这个例子中，考虑到我们所有的数据量（n=9），我们不能超过的数据点的个数，大于n的都不行，对它们的约束不够。  
 在这个例子中，最大的order是8.
 
 ## Polynomial Regression 多项式回归：
 ![title](../images/polynomial.png)
 
 上图用矩阵形式表达为：
 $Xw\thickapprox{y}$    
 我们基本上只需要对$w$这个向量求解，现在我们无法真正对它求解，因为上面这个式子并不是完全相等的，但是我们可以用最小二乘法思想来求解。  
 下面逐步完成所需的每个步骤：   
 ![tilte](../images/w_1.png)
 
 正好这些x转置和x有一些不错的属性，$(X^{T}X)^{-1}$不仅可逆，它还可以最小化最小二乘。  它的作用是射影，现在我们将不对它的正确性进行证明。
 
 ## Errors:
 我们无法通过解线性方程组来解答这类问题的部分原因，
 我们不得不使用这些平方计算的原因是**存在误差**，就是提供给我们的训练数据存在误差。
 并不是我们要真正建模的不是函数，而我们见到的函数加上每条数据上的误差项。
 
 Training data has errors.    
 Not modeling f, but fits.    
 
 
 我认为有理由想一想这些误差是如何产生的。  
 为什么我们要试图拟合有误差的数据？
 
 Where do errors come from?
 - sensor error
 - maliciously - being given bad data
 - transcription error
 - unmodeled influences
 
 
 ## Cross Validation:
 我们的目的始终是泛化generalize。
 
 ## Conclusion：
 What did we learn?
 - historial facts
 - model selection and under/overfitting
 - cross validation
 - linear, Polynomial regression
 - best constant in terms of squared error is mean
 - representation for regression
 

## sklearn
sklearn.linear_model.LinearRegression

官方示例：

In [19]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

print('slope: ', reg.coef_)
print('intercept: ', reg.intercept_)
print('predicted: ', reg.predict([[3, 3]]))
print('r-squared score: ', reg.score([[4, 4], [5, 5]],  [4, 5]))

slope:  [0.5 0.5]
intercept:  2.220446049250313e-16
predicted:  [3.]
r-squared score:  1.0
